In [ ]:

##################################################### Packages ###################################################################
import os
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb

import torch
from torch.utils.data import DataLoader


from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Training class

In [ ]:


# class TrainViT:
#     """
#     Class to pretrain the generator network
#     """
#     def __init__(self, 
#                  image_size = 224, 
#                  batch_size = 32, 
#                  epochs = 100, 
#                  lr = 0.0002, 
#                  beta1 = 0.5, 
#                  beta2 = 0.999, 
#                  weight_decay=0, 
#                  loss = nn.CosineSimilarity(), 
#                  run = "training_run",
#                  save_dir = '/results', 
#                  start_epoch = 0):
#         """
#         Initializes class TrainViT: class with all default values.
#         See methods to perform sets.
#         """
#         self.image_size = image_size
#         self.batch_size = batch_size
#         self.lr = lr
#         self.beta1 = beta1
#         self.beta2 = beta2
#         self.weight_decay = weight_decay
#         self.loss = loss
#         self.run = run
#         self.save_dir = save_dir
#         self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#         self.model = None
#         self.optimizer = None
#         self.scheduler = None

#         self.start_epoch = start_epoch
#         self.epochs = epochs
#         self.avg_loss = 0

#         self.train_ds = None
#         self.val_ds = None
#         self.train_dl = None
#         self.val_dl = None

#         self.train_loss = []
#         self.val_loss = []

#         self.val_paths = None
#         self.train_paths = None

#     def set_train_and_val_paths(self, data_dir:str, num_images:int) -> None:
#         """
#         Implement me
#         """
#         self.train_paths, self.val_paths = 
        
#     def set_model(self, model:callable = None) -> None:
#         """
#         Set the generator model and optimizer, default is to use a U-Net with a ResNet18 backbone
#         """
#         self.model = model

#     def load_state(self, path_to_checkpoint:str) -> None:
#         """
#         Loads a previous model state
#         """
#         try:
#             checkpoint = torch.load(path_to_checkpoint)
#             self.model.load_state_dict(checkpoint["model_state_dict"])
#             self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#             self.start_epoch = checkpoint['epoch']
#             self.avg_loss = checkpoint['loss']
#             print(f"Model state loaded successfully!")
#         except FileNotFoundError as e:
#             print("Error loading generator weights!")
#         return

#     def set_data_loaders(self, perform_checks:bool = True) -> None:
#         """
#         Set up the dataloaders
#         """
#         self.train_ds = IsdDataSet(self.batch_size, paths = self.train_paths, split = "train")
#         self.val_ds = IsdDataSet(self.batch_size, paths = self.val_paths, split = "val")
#         self.train_dl = DataLoader(self.train_ds, batch_size = self.batch_size)
#         self.val_dl = DataLoader(self.val_ds, batch_size = self.batch_size)

#         if perform_checks:
#             data = next(iter(self.train_dl))
#             imgs, isds = data['L'], data['ab']
#             assert imgs.shape == isds.shape 
#             print(f" Image data dims: {imgs.shape} \nISD data dims: {isds.shape}")
#             print(len(self.train_dl), len(self.val_dl))

#         return

#     def train_loop(self, epoch) -> None:
#         """
#         Performs the train loop tracking train loss
#         """
#         epoch_train_loss = 0
#         num_batches = 0

#         # Train Loop
#         pbar = tqdm(self.train_dl, desc=f"Training Epoch {self.start_epoch}/{self.start_epoch + self.epochs}")
#         for i, data in enumerate(pbar):
#             imgs, isds = data["imgs"], data["isds"]
#             imgs, isds = imgs.to(self.device), isds.to(self.device)
    
#             # Train the generator
#             self.model.train()
#             self.optimizer.zero_grad()
#             model_output = self.model(imgs)
    
#             LOSS = self.loss(model_output, isds) 
#             LOSS.backward()
#             self.optimizer.step()
    
#             # Accumulate losses
#             epoch_train_loss += LOSS.item()
#             num_batches += 1
    
#             # Update progress bar with current loss values
#             pbar.set_postfix(G_loss=LOSS.item())
    
#         # Average losses for the epoch
#         avg_train_loss = epoch_train_loss / num_batches
#         self.train_loss.append(avg_train_loss)
#         print(f"The average loss for epoch: {epoch} - {avg_train_loss}")
    
#         self.scheduler.step(avg_train_loss)

#     def val_loop(self, epoch) -> None:
#         """
#         Performs the val loop tracking val loss
#         """
#         with torch.no_grad():
#             num_batches = 0
#             epoch_val_loss = 0
#             self.model.eval()
            
#             pbar = tqdm(self.val_dl, desc=f"Validation Epoch {self.start_epoch}/{self.start_epoch + self.epochs}")
#             for i, data in enumerate(pbar):
#                 imgs, isds = data["imgs"], data["isds"]
#                 imgs, isds = imgs.to(self.device), isds.to(self.device)
    
#                  # Evaluate the generator
#                 model_output = self.model(imgs)
#                 LOSS = self.loss(model_output, isds) 
        
#                 # Accumulate losses
#                 epoch_val_loss += LOSS.item()
#                 num_batches += 1
        
#                 # Update progress bar with current loss values
#                 pbar.set_postfix(G_loss=LOSS.item())
    
#                 # Create the directory to save iamges in
#                 image_save_dir = f"{str(Path.cwd())}/training_runs/{self.run}/val_images/"
#                 os.makedirs(image_save_dir, exist_ok=True)
#                 image_save_path = image_save_dir + f"epoch_{epoch}.png"
                
#                 if epoch % 10 == 0:
#                     self.plot_batch(imgs, model_output, isds, show = False, save_path = image_save_path)
        
#         # Calculate average validation loss
#         avg_val_loss = epoch_val_loss / num_batches
#         self.val_loss.append(avg_val_loss)
#         print(f"Avg Validation Loss: {avg_val_loss}")
#         return

#     def plot_losses(self, epoch) -> None:
#         """
#         Generates and saves loss versus epoch plot
#         """
#         # Create fig
#         figs_save_dir = f"{str(Path.cwd())}/training_runs/{self.run}/loss_figs/"
#         os.makedirs(figs_save_dir, exist_ok=True)
#         figs_save_path = figs_save_dir + f"epoch_{epoch}.png"
                
#         # Ensure the directory exists
#         epoch_range = range(self.start_epoch, epoch + 1)
#         plt.plot(epoch_range, self.train_loss_generator, c = "b", label = "Train Loss")
#         plt.plot(epoch_range, self.val_loss_generator, c = "r", label = "Val Loss")
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(figs_save_path)
#         plt.close

#     def save_model_state(self, epoch:int) -> None:
#         """
#         Saves the current model state
#         """
#         # Path to model weights location 
#         state_save_dir = f"{self.save_dir}/model_states"
    
#         # Ensure the directory exists
#         os.makedirs(state_save_dir, exist_ok=True)
    
#         # Update path with file name
#         state_save_path = os.path.join(state_save_dir, f'checkpoint_epoch_{epoch}.pth')
    
#         # Save the model weights
#         torch.save({
#             'epoch': epoch,
#             'model_state_dict': self.model.state_dict(),
#             'optimizer_state_dict': self.optimizer.state_dict(),
#             'loss': self.train_loss_generator[-1],
#         }, state_save_path)
#         print(f"Model state saved to: {state_save_path}")

    
#     def save_config(self, config, file_name="config.yaml"):
#         """
#         Save the configuration dictionary as a YAML file.

#         Args:
#             config (dict): The configuration parameters to save.
#             file_name (str): The name of the YAML file. Defaults to "config.yaml".
#         """
#         save_path = os.path.join(self.save_dir, file_name)
#         try:
#             with open(save_path, "w") as yaml_file:
#                 yaml.dump(config, yaml_file, default_flow_style=False)
#             print(f"Configuration saved to {save_path}")
#         except Exception as e:
#             print(f"Error saving configuration: {e}")
        
#     def train_model(self) -> None:
#         """
#         Trains the model
#         """
#         for epoch in range(self.start_epoch, self.start_epoch + self.epochs):
#             self.train_loop(epoch)
#             self.val_loop(epoch)
#             if epoch % 10 == 1:
#                 self.plot_losses()
#                 self.save_model_state()
        
#     def set_optimizer(self, model_params) -> None:
#         """
#         Method to set up the optimizer
#         """
#         self.optimizer = torch.optim.Adam(model_params, lr=self.lr, betas=(self.beta1, self.beta2), weight_decay=self.weight_decay)

#     def set_scheduler(self) -> None:
#         """
#         Method to set up the scheduler
#         """
#         raise NotImplementedError

# print("This works!!")
# if __name__ == "__main__":
#     pass


### Param YAML 

In [ ]:
# General training configuration
checkpoint_path: null       # Path to checkpoint file if loading a previous state
load_previous_state: false  # Whether to load a previous state
data_dir: "path/to/data"    # Directory containing .tif images
num_images: 1000            # Number of images to use for training
size: 224                   # Image size
batch_size: 32              # Batch size for training
epochs: 101                 # Number of epochs for training
lr: 0.0002                  # Learning rate
beta1: 0.5                  # Beta1 for Adam optimizer
beta2: 0.999                # Beta2 for Adam optimizer
run: "test_run_01"          # Run name for logging and saving checkpoints
save_dir: "/results/test_run_01"  # Explicitly set save directory
start_epoch: 0              # Starting epoch

# Optimizer configuration
optimizer:
  type: "Adam"              # Type of optimizer (e.g., "Adam", "SGD", etc.)
  lr: 0.0002                # Learning rate
  weight_decay: 0.0001      # Weight decay (L2 regularization)
  beta1: 0.5                # Beta1 parameter for Adam
  beta2: 0.999              # Beta2 parameter for Adam
  momentum: null            # Momentum (only used for optimizers like SGD)

# Scheduler configuration (choose one)
scheduler:
  use_scheduler: true       # Whether to use a learning rate scheduler
  type: "StepLR"            # Type of scheduler ("StepLR", "ReduceLROnPlateau", etc.)
  params:                   # Scheduler-specific parameters
    step_size: 10           # For StepLR
    gamma: 0.1              # For StepLR

# Alternative scheduler configurations (comment out the above and uncomment one of these if needed)
# scheduler:
#   use_scheduler: true
#   type: "ReduceLROnPlateau"
#   params:
#     mode: "min"
#     factor: 0.1
#     patience: 5
#     threshold: 0.01
#     cooldown: 2

# scheduler:
#   use_scheduler: true
#   type: "CosineAnnealingLR"
#   params:
#     T_max: 50
#     eta_min: 1e-6

# Pretrained settings
pretrained:
  checkpoint_path: null     # Path to checkpoint file if loading a previous state
  load_model_state: false   # Whether to load the model state
  load_optim_states: false  # Whether to load optimizer states

### Train driver

In [ ]:
# import glob
# import yaml

# def load_optimizer_states(optimizer, config):
#     """
#     Loads the optimizer states for generator and discriminator from the checkpoint
#     if they exist and if the config specifies to do so.

#     Args:
#         optimizer (torch.optim.Optimizer): Optimizer
#         checkpoint (dict): The loaded checkpoint containing model and optimizer states.
#         config (dict): Configuration dictionary specifying whether to load optimizer states.

#     Returns:
#         None
#     """
#     if config['pretrained']['checkpoint_path']:
#         checkpoint_path = config['pretrained']['checkpoint_path']
#         checkpoint = torch.load(checkpoint_path)
        
#     if config['pretrained']['load_optim_states']:
#         try:
#             if 'optimizer_state_dict' in checkpoint:
#                 optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#             else:
#                 print("Optimizer state dictionaries not found in checkpoint.")
#         except Exception as e:
#             print(f"Error loading optimizer states: {e}")

#     return optimizer


# # Load parameters from YAML
# with open("config.yaml", "r") as file:
#     params = yaml.safe_load(file)

# # Extract parameters
# checkpoint_path = params.get("checkpoint_path")
# load_previous_state = params.get("load_previous_state", False)
# data_dir = params["data_dir"]
# paths = glob.glob(data_dir + "/*.tif")
# num_images = params["num_images"]
# size = params["size"]
# batch_size = params["batch_size"]
# epochs = params["epochs"]
# lr = params["lr"]
# beta1 = params["beta1"]
# beta2 = params["beta2"]
# l1_loss = nn.L1Loss()
# run = params["run"]
# save_dir = params["save_dir"]
# start_epoch = params["start_epoch"]

# # Select model
# model = None  # Define or load your model here if needed
# model_params = model.parameters()

# # Train model
# vit_trainer = TrainViT(size, batch_size, epochs, lr, beta1, beta2, l1_loss, run, start_epoch)
# vit_trainer.set_train_and_val_paths(paths, num_images)
# vit_trainer.set_data_loaders()
# vit_trainer.set_model(model=model)
# vit_trainer.set_optimizer(model_params=model_params)
# if load_previous_state:
#     vit_trainer.load_state(checkpoint_path)
# vit_trainer.save_config(config=params)    
# vit_trainer.train_model()

In [ ]:
import os
import logging
import torch
from torch import nn
from torch.utils.data import DataLoader
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import yaml

from dataloader_dev import ImageDatasetGenerator

class TrainViT:
    """
    Class to pretrain the generator network with a Vision Transformer backbone.
    """

    def __init__(self,
                 image_size=224,
                 batch_size=32,
                 epochs=100,
                 lr=0.0002,
                 beta1=0.5,
                 beta2=0.999,
                 weight_decay=0,
                 loss=nn.CosineSimilarity(),
                 run="training_run",
                 save_dir='/results',
                 start_epoch=0):
        """
        Initializes TrainViT class with default or user-provided values.
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.setLevel(logging.INFO)

        self.image_size = image_size
        self.batch_size = batch_size
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.weight_decay = weight_decay
        self.loss = loss
        self.run = run
        self.save_dir = save_dir
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model = None
        self.optimizer = None
        self.scheduler = None

        self.start_epoch = start_epoch
        self.epochs = epochs
        self.avg_loss = 0

        self.train_ds = None
        self.val_ds = None
        self.train_dl = None
        self.val_dl = None

        self.train_loss = []
        self.val_loss = []

        self.val_paths = None
        self.train_paths = None

    def set_train_and_val_paths(self, data_dir: str, num_images: int) -> None:
        """
        Placeholder for setting training and validation paths.
        """
        self.train_paths = os.path.join(data_dir, "train")
        self.val_paths = os.path.join(data_dir, "val")
        self.logger.info(f"Train and val paths set: {self.train_paths}, {self.val_paths}")

    def set_model(self, model: callable = None) -> None:
        """
        Sets the generator model.
        """
        self.model = model.to(self.device)
        self.logger.info("Model initialized.")

    def load_state(self, path_to_checkpoint: str) -> None:
        """
        Loads a previous model state.
        """
        try:
            checkpoint = torch.load(path_to_checkpoint, map_location=self.device)
            self.model.load_state_dict(checkpoint["model_state_dict"])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.start_epoch = checkpoint['epoch']
            self.avg_loss = checkpoint['loss']
            self.logger.info("Model state loaded successfully.")
        except FileNotFoundError as e:
            self.logger.error(f"Error loading checkpoint: {e}")
        except KeyError as e:
            self.logger.error(f"Checkpoint is missing a key: {e}")

    def set_data_loaders(self, image_dir: str, isd_map_dir: str, perform_checks: bool = True) -> None:
        """
        Sets up the dataloaders.

        Args:
            image_dir (str): Directory of training images.
            isd_map_dir (str): Directory of isd maps corresponding to training images. 
            performs_check (bool): If True (default), will check that dataloader output.

        Returns:
            None
        """
        # Transforms -- We can add augmentations here
        transform_images = transforms.Compose([ 
                                transforms.ToTensor()       
                                #transforms.Normalize(mean = mean, std = std)
                                            ])

        transform_guidance = transforms.Compose([ 
                                transforms.ToTensor()
                                            ])
        
        # Create train split
        self.train_ds = ImageDatasetGenerator(image_dir, 
                                            isd_map_dir, 
                                            split = "train", 
                                            val_size = 0.2, 
                                            random_seed = 42, 
                                            transform_images = transform_images, 
                                            transform_guidance = transform_guidance)
        self.train_dl = DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=True, drop_last=True)

        # Create val split
        self.val_ds = ImageDatasetGenerator(image_dir, 
                                            isd_map_dir, 
                                            split = "val", 
                                            val_size = 0.2, 
                                            random_seed = 42, 
                                            transform_images = transform_images, 
                                            transform_guidance = transform_guidance)
        self.val_dl = DataLoader(self.val_ds, batch_size=self.batch_size, shuffle=True, drop_last=True)

        if perform_checks:
            if not len(self.train_ds):
                self.logger.warning("The dataloader is empty. No batches to inspect.")
                return None, None

            for idx, (images, isds) in enumerate(self.train_ds):
                self.logger.info(f"Inspecting Batch {idx + 1}")
                self.logger.info(f"Images shape: {images.shape}")
                self.logger.info(f"ISDs shape: {isds.shape}")
                if idx == 0:
                    break

    def set_optimizer(self, model_params, optimizer_type="Adam", **kwargs) -> None:
        """
        Method to set up the optimizer.

        Args:
            model_params (iterable): Parameters of the model to optimize.
            optimizer_type (str): Type of optimizer. Default is 'Adam'.
            **kwargs: Additional parameters for the optimizer.
        """
        if optimizer_type == "Adam":
            self.optimizer = torch.optim.Adam(
                model_params, 
                lr=self.lr, 
                betas=(self.beta1, self.beta2), 
                weight_decay=self.weight_decay,
                **kwargs
            )
        elif optimizer_type == "SGD":
            self.optimizer = torch.optim.SGD(
                model_params, 
                lr=self.lr, 
                momentum=kwargs.get("momentum", 0.9), 
                weight_decay=self.weight_decay,
                **kwargs
            )
        else:
            raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

        logging.info(f"Optimizer set to {optimizer_type} with lr={self.lr}, weight_decay={self.weight_decay}")

    def set_scheduler(self, scheduler_type="StepLR", **kwargs) -> None:
        """
        Method to set up the learning rate scheduler.

        Args:
            scheduler_type (str): Type of learning rate scheduler. Default is 'StepLR'.
            **kwargs: Additional parameters for the scheduler.
        """
        if not self.optimizer:
            raise ValueError("Optimizer must be set before defining the scheduler.")

        if scheduler_type == "StepLR":
            self.scheduler = torch.optim.lr_scheduler.StepLR(
                self.optimizer, 
                step_size=kwargs.get("step_size", 10), 
                gamma=kwargs.get("gamma", 0.1)
            )
        elif scheduler_type == "ReduceLROnPlateau":
            self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, 
                mode=kwargs.get("mode", "min"), 
                factor=kwargs.get("factor", 0.1), 
                patience=kwargs.get("patience", 10)
            )
        elif scheduler_type == "CosineAnnealingLR":
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer, 
                T_max=kwargs.get("T_max", 50), 
                eta_min=kwargs.get("eta_min", 0)
            )
        else:
            raise ValueError(f"Unsupported scheduler type: {scheduler_type}")

        logging.info(f"Scheduler set to {scheduler_type} with parameters: {kwargs}")

    def train_loop(self, epoch: int) -> None:
        """
        Performs the training loop for a single epoch.
        """
        epoch_train_loss = 0
        num_batches = 0

        self.model.train()
        pbar = tqdm(self.train_dl, desc=f"Training Epoch {epoch}/{self.epochs}")
        for i, data in enumerate(pbar):
            imgs, isds = data["imgs"].to(self.device), data["isds"].to(self.device)

            self.optimizer.zero_grad()
            model_output = self.model(imgs)
            loss = self.loss(model_output, isds)
            loss.backward()
            self.optimizer.step()

            epoch_train_loss += loss.item()
            num_batches += 1
            pbar.set_postfix(G_loss=loss.item())

        avg_train_loss = epoch_train_loss / num_batches
        self.train_loss.append(avg_train_loss)
        self.logger.info(f"Epoch {epoch} - Average Training Loss: {avg_train_loss}")
        return avg_train_loss
    
    def val_loop(self, epoch: int) -> None:
        """
        Performs the validation loop for a single epoch.
        """
        with torch.no_grad():
            epoch_val_loss = 0
            num_batches = 0

            self.model.eval()
            pbar = tqdm(self.val_dl, desc=f"Validation Epoch {epoch}/{self.epochs}")
            for i, data in enumerate(pbar):
                imgs, isds = data["imgs"].to(self.device), data["isds"].to(self.device)

                model_output = self.model(imgs)
                loss = self.loss(model_output, isds)

                epoch_val_loss += loss.item()
                num_batches += 1
                pbar.set_postfix(Val_loss=loss.item())

        avg_val_loss = epoch_val_loss / num_batches
        self.val_loss.append(avg_val_loss)
        self.logger.info(f"Epoch {epoch} - Average Validation Loss: {avg_val_loss}")

    def save_model_state(self, epoch: int) -> None:
        """
        Saves the current model state to a checkpoint.
        """
        state_save_dir = os.path.join(self.save_dir, "model_states")
        os.makedirs(state_save_dir, exist_ok=True)

        save_path = os.path.join(state_save_dir, f'checkpoint_epoch_{epoch}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'loss': self.train_loss[-1],
        }, save_path)
        self.logger.info(f"Model state saved at epoch {epoch}: {save_path}")

    def save_config(self, config: dict, file_name="config.yaml") -> None:
        """
        Saves the configuration dictionary to a YAML file.
        """
        save_path = os.path.join(self.save_dir, file_name)
        try:
            with open(save_path, "w") as yaml_file:
                yaml.dump(config, yaml_file, default_flow_style=False)
            self.logger.info(f"Configuration saved to {save_path}")
        except Exception as e:
            self.logger.error(f"Error saving configuration: {e}")

    def plot_losses(self, epoch: int) -> None:
        """
        Plots and saves the loss vs epoch graph.
        """
        figs_save_dir = os.path.join(self.save_dir, "loss_figs")
        os.makedirs(figs_save_dir, exist_ok=True)

        plt.figure()
        plt.plot(range(self.start_epoch, epoch + 1), self.train_loss, label="Train Loss", color="b")
        plt.plot(range(self.start_epoch, epoch + 1), self.val_loss, label="Validation Loss", color="r")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.tight_layout()
        fig_path = os.path.join(figs_save_dir, f"loss_epoch_{epoch}.png")
        plt.savefig(fig_path)
        plt.close()
        self.logger.info(f"Loss plot saved: {fig_path}")

    def train_model(self) -> None:
        """
        Trains the model over multiple epochs and returns the final model, optimizer states, and loss history.
        """
        for epoch in range(self.start_epoch, self.start_epoch + self.epochs):
            self.train_loop(epoch)
            val_loss = self.val_loop(epoch)

            # Scheduler step
            if isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step(val_loss)
            else:
                self.scheduler.step()

            if epoch % 10 == 0 or epoch == self.epochs - 1:
                self.plot_losses(epoch)
                self.save_model_state(epoch)

        return {
        "model_state_dict": self.model.state_dict(),
        "optimizer_state_dict": self.optimizer.state_dict(),
        "train_loss_history": self.train_loss,
        "val_loss_history": self.val_loss
        }

In [ ]:
import os
import yaml
import glob
import torch
from torch import nn
import logging

from transformer_dev import VisionTransformer, TrainViT


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Starting training...")

def load_optimizer_states(optimizer, config):
    checkpoint_path = config['pretrained']['checkpoint_path']
    if config['pretrained']['load_optim_states'] and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        if 'optimizer_state_dict' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        else:
            logger.warning("Optimizer state dictionaries not found in checkpoint.")
    return optimizer

# Load YAML configuration
try:
    with open("config.yaml", "r") as file:
        params = yaml.safe_load(file)
except FileNotFoundError:
    raise FileNotFoundError("The config.yaml file was not found. Please provide a valid path.")
except yaml.YAMLError as e:
    raise ValueError(f"Error parsing YAML file: {e}")

# Validate required keys
required_keys = ["data_dir", "num_images", "batch_size", "epochs", "lr", "beta1", "beta2", "run", "save_dir"]
for key in required_keys:
    if key not in params:
        raise ValueError(f"Missing required configuration key: {key}")

# Extract parameters
data_dir = params["data_dir"]
paths = glob.glob(os.path.join(data_dir, "*.tif"))
num_images = params["num_images"]
size = params["size"]
batch_size = params["batch_size"]
epochs = params["epochs"]
lr = params["lr"]
beta1 = params["beta1"]
beta2 = params["beta2"]
run = params["run"]
save_dir = os.path.abspath(params["save_dir"])
start_epoch = params.get("start_epoch", 0)
l1_loss = nn.CosineSimilarity()

# Model setup
model = VisionTransformer(num_layers=12, img_size=size, embed_dim=768, patch_size=16, num_head=8, cnn_embedding=True)
model_params = model.parameters()

# Initialize training class
vit_trainer = TrainViT(size, batch_size, epochs, lr, beta1, beta2, l1_loss, run, start_epoch)
vit_trainer.set_train_and_val_paths(paths, num_images)
vit_trainer.set_data_loaders()
vit_trainer.set_model(model=model)
vit_trainer.set_optimizer(model_params=model_params)

# Load checkpoint if required
if params["pretrained"]["load_model_state"]:
    vit_trainer.load_state(params["pretrained"]["checkpoint_path"])

# Save configuration and start training
vit_trainer.save_config(config=params)
logger.info(f"Starting Training")
results_dict  = vit_trainer.train_model()

logger.info(f"Final training loss: {results_dict['train_loss_history'][-1]}")
logger.info(f"Final validation loss: {results_dict['train_loss_history'][-1]}")